# Topological Associated Domain boundary prediction

This notebook aims to reproduce results of the recent paper, [Henderson et al, 2019](https://academic.oup.com/nar/advance-article/doi/10.1093/nar/gkz315/5485073#supplementary-data), where the group was able to predict high resolution Topologically Associated Domain (TAD) boundaries from sequencing data using a convolutional neural network. After model fine-tuning and optimization they were able to detect boundaries with an accuracy of 96% from one-hot encoded sequencing data alone. Here I will attempt to reproduce their results using the [fastai](https://docs.fast.ai/) library, built on [PyTorch](https://pytorch.org/). 

> A, T, C and G were encoded as [1, 0, 0,
0], [0, 1, 0, 0], [0, 0, 1, 0] and [0, 0, 0, 1],

> Each sequence is converted to a matrix of shape (L,4).

1D convolutions using kernels of shape (k, 4). Yilded activation map ((L+2(p – d)(k – 1) – 1)/s, 4). 

In [1]:
%reload_ext autoreload
%autoreload 2 
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *
import h5py

Data were cloned from the public repository, https://github.com/lincshunter/TADBoundaryDectector and unpacked with the `unrar` utility in the data directory.

In [3]:
dpath = Path("data/TADBoundaryDectector")

In [4]:
dpath.ls()

[PosixPath('data/TADBoundaryDectector/.git'),
 PosixPath('data/TADBoundaryDectector/Models.py'),
 PosixPath('data/TADBoundaryDectector/README.md'),
 PosixPath('data/TADBoundaryDectector/dm3.kc167.example.rar'),
 PosixPath('data/TADBoundaryDectector/dm3.kc167.example.h5')]

In [5]:
h5file = "data/TADBoundaryDectector/dm3.kc167.example.h5"
f = h5py.File(h5file, 'r') 

In [6]:
f.keys() # list available keys

<KeysViewHDF5 ['x_test', 'x_train', 'x_val', 'y_test', 'y_train', 'y_val']>

In [7]:
x_test = np.array(f['x_test'])
x_test.shape # a 1k by 1k by 4 tensor

(1000, 1000, 4)

In [8]:
x_train = np.array(f['x_train'])
x_val = np.array(f['x_val']) 
y_test = np.array(f['y_test'])
y_train = np.array(f['y_train'])
y_val = np.array(f['y_val'])

In [9]:
 x_test.shape

(1000, 1000, 4)

In [10]:
x_val.shape

(1000, 1000, 4)

In [11]:
y_test.shape

(1000,)

In [12]:
y_train.shape

(28127,)

In [13]:
y_val.shape

(1000,)

Now let's cooerce these data into a fastai databunch object. Convert all to tensors. 

In [17]:
x_test,x_train,x_val,y_test,y_train,y_val = map(torch.tensor, [x_test,x_train,x_val,y_test,y_train,y_val])

In [18]:
x_test.shape

torch.Size([1000, 1000, 4])

In [19]:
# use minibatches
bs=50
train_ds = TensorDataset(x_train, y_train)
valid_ds = TensorDataset(x_val, y_val)
test_ds = TensorDataset(x_test, y_test)
data = DataBunch.create(train_ds, valid_ds, test_ds, bs=bs)

In [20]:
x,y=next(iter(data.train_dl))
x.shape, y.shape

(torch.Size([50, 1000, 4]), torch.Size([50]))

In [21]:
x,y=data.one_batch()
y,x.shape

(tensor([1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
         0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
         0, 1]), torch.Size([50, 1000, 4]))

In [22]:
# batch, chan, len
x.shape[1]
x.transpose(1,2).shape

torch.Size([50, 4, 1000])

Create a model architecture like [three_CNN_LSTM](https://github.com/lincshunter/TADBoundaryDectector/blob/master/Models.py)


pseudocode of the three_CNN_LSTM model:


 - sequential
 - conv1d
 - relU
 - dropout(0.2)
 - maxpool1d


 - conv1d
 - relu
 - relu
 - relu
 - dropout 0.3
 - maxpool1d 


 - conv1d
 - relu
 - dropout(0.3)
 - maxpool1d 


 - bidirectionalLSTM
 - flatten 
 - dense
 - sigmoid


using ADAM optimizaiton with cross entropy loss



This might be helpful 

https://towardsdatascience.com/understanding-bidirectional-rnn-in-pytorch-5bd25a5dd66





In [23]:
# INIT PARAMETERS
s = 1 # stride 
p = 1 # padding 
d = 0 # dilation
bs = 50 # batch size 
lr = 1e-3 
inshape = x_train.shape[1:3] 
ks = 9 # kernel size
nk = 64 # n kernels
lstmU = 40 # LSTM units

((L+2(p – d)(k – 1) – 1)/s, 4), the resulting size of a 1d convolution on the input. 

In [37]:
(1000+2*(9-1) - 1, 4) # after first conv1d

(1015, 4)

In [38]:
(1015+2*(9-1) - 1, 4) # after second conv1d

(1030, 4)

In [40]:
(1030+2*(9-1) - 1, 4) # after thrid conv1d

(1045, 4)

In [24]:
print(inshape)

torch.Size([1000, 4])


In [365]:
# import torch.nn.functional as F

# epochs = 150 
# kernsize = 9 
# bs = 50
# verbose = 1 
# nclass = 2
# metrics = ['accuracy']
# loss = 'binary_crossentropy'
# kernel = 'glorot_uniform'

class BoundaryModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(4,nk,ks,padding=1) # inchan # outchan # len 
        self.drop1 = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size = 5,
                                 stride = 2)
        
        self.conv2 = nn.Conv1d(64,nk,ks,padding=1) 
        self.drop2 = nn.Dropout(p=0.3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size = 5,
                                 stride = 2)
        
        # need to have this up here not in the forward
        self.bgru = nn.GRU(117, 40, bidirectional=True)
        self.lin = nn.Linear(64,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):

        # permute shape from [batch, len, chan] to [batch, chan, len]
        x = torch.Tensor.permute(x,0,2,1)
        
        x = self.conv1(x)
        x = self.relu(x)
        x = self.drop1(x)
        x = self.pool(x)
        
        print(f"Size after first pool is {x.size()}")
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.drop2(x)
        x = self.pool(x)
        
        print(f"Size after second pool is {x.size()}")
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.drop2(x)
        x = self.pool(x)
        
        print(f"Size after third pool is {x.size()}")
        
        _, hn = self.bgru(x)
        print(f"Size after bidirectional GRU is {hn.size()}")
        x = hn.view(-1, x.size(1))
        print(f"Size after flatten is {x.size()}")
        x = self.lin(x)
        x = self.relu(x)
        
        print(f"Final Size: {x.size()}")
        
        return self.sigmoid(x)
    

In [366]:
m = BoundaryModel().cuda()
data1_mod = m(x.type(torch.FloatTensor).cuda())

Size after first pool is torch.Size([50, 64, 495])
Size after second pool is torch.Size([50, 64, 243])
Size after third pool is torch.Size([50, 64, 117])
Size after bidirectional GRU is torch.Size([2, 64, 40])
Size after flatten is torch.Size([80, 64])
Final Size: torch.Size([80, 1])


In [352]:
data1_mod.shape

torch.Size([80, 1])

In [353]:
learn = Learner(data, m, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)

In [354]:
learn.summary

<bound method model_summary of Learner(data=DataBunch;

Train: <torch.utils.data.dataset.TensorDataset object at 0x7f55bab4e470>;

Valid: <torch.utils.data.dataset.TensorDataset object at 0x7f55bab4e4a8>;

Test: <torch.utils.data.dataset.TensorDataset object at 0x7f55bab4e4e0>, model=BoundaryModel(
  (conv1): Conv1d(4, 64, kernel_size=(9,), stride=(1,), padding=(1,))
  (drop1): Dropout(p=0.2)
  (relu): ReLU()
  (pool): MaxPool1d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(1,))
  (drop2): Dropout(p=0.3)
  (bgru): GRU(117, 40, bidirectional=True)
  (lin): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
), opt_func=functools.partial(<class 'torch.optim.adam.Adam'>, betas=(0.9, 0.99)), loss_func=CrossEntropyLoss(), metrics=[<function accuracy at 0x7f55d269cb70>], true_wd=True, bn_wd=True, wd=0.01, train_bn=True, path=PosixPath('.'), model_dir='models', callback_fns=[functools.par